In [1]:
import requests
from bs4 import BeautifulSoup

In [40]:
class Team:
    _id: str
    name: str
    division: str

    def __init__(self, url):
        self._id = url.split('/')[-2]
        self.get_team_info(url)

    def is_in_db(self):
        
    def scrap_team_info(self, url):
        self._id = url.split('/')[-2]
        page = requests.get(url)
        soup = BeautifulSoup(page.content, "html.parser")
        page_html = soup.find(id="fitt-analytics").find(id="fittPageContainer")
        name_html = page_html.find("h1", class_="ClubhouseHeader__Name ttu flex items-start n2").find("span")
        self.name = ' '.join([span.text for span in name_html.find_all("span")])
        division_html = page_html.find("ul", class_="list flex ClubhouseHeader__Record n8 ml4").find_all("li")[-1]
        self.division = division_html.text.split("in ")[-1]

In [41]:
url = "https://www.espn.com/nba/scoreboard/_/date/20221119"
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")
games_html = soup.find(id="fitt-analytics").find(id="fittPageContainer").find_all(class_="Scoreboard bg-clr-white flex flex-auto justify-between")

In [42]:
games_html

[<section class="Scoreboard bg-clr-white flex flex-auto justify-between" id="401468390"><div class="Scoreboard__RowContainer flex flex-column flex-auto"><div class="Scoreboard__Row flex w-100 Scoreboard__Row__Main"><div class="Scoreboard__Column flex-auto Scoreboard__Column--1 Scoreboard__Column--Score" style="min-width:320px"><div class=""><div class="ScoreboardScoreCell pa4 nba basketball ScoreboardScoreCell--post ScoreboardScoreCell--tabletPlus"><div class="ScoreboardScoreCell__Overview flex pb3 w-100"><div class="ScoreCell__Time ScoreboardScoreCell__Time h9 clr-gray-01"></div><div class="ScoreboardScoreCell__Headings flex justify-end"><div class="ScoreboardScoreCell__Heading flex justify-center pl2 h9">1</div><div class="ScoreboardScoreCell__Heading flex justify-center pl2 h9">2</div><div class="ScoreboardScoreCell__Heading flex justify-center pl2 h9">3</div><div class="ScoreboardScoreCell__Heading flex justify-center pl2 h9">4</div><div class="ScoreboardScoreCell__Heading flex jus

In [45]:
PLAYING_HOME_INDEX = 5
WINNER_INDEX = 6
for game_html in games_html:
    game_id = game_html['id']
    c = game_html.find("ul", class_="ScoreboardScoreCell__Competitors").find_all("li")
    print(len(c))
    for t in c:
        class_name = t["class"]
        is_playing_home = True if 'home' in class_name[PLAYING_HOME_INDEX] else False
        is_winner = True if 'winner' in class_name[WINNER_INDEX] else False    
        team_link = t.find("a", class_="AnchorLink truncate")["href"]
        team = Team(f"https://www.espn.com{team_link}")
        score = t.find("div", class_="ScoreCell__Score h4 clr-gray-01 fw-heavy tar ScoreCell_Score--scoreboard pl2").text
        print(score)
        if is_playing_home:
            team_home_id = team._id
            team_home_score = score
        else:
            team_away_id = team._id
            team_away_score = score
        
        print(team_link)
    game = Game2(game_id, date, team_home_id, team_away_id, team_home_score, team_away_score)
    
    break

2
122
/nba/team/_/name/tor/toronto-raptors
124
/nba/team/_/name/atl/atlanta-hawks


NameError: name 'Game2' is not defined

In [2]:
BOXSCORE_KEY = "Boxscore flex flex-column"
TEAM_NAME = "BoxscoreItem__TeamName h5"
TEAM_TAB = "ResponsiveTable ResponsiveTable--fixed-left Boxscore flex flex-column"
TABLE_BODY = "Table__TBODY"
TABLE_CONTENT = "Table__TR Table__TR--sm Table__even"
PLAYER = "Table Table--align-right Table--fixed Table--fixed-left"
PLAYER_NAME = "AnchorLink truncate db external Boxscore__AthleteName"
PLAYER_POSITION = "playerPosition pl2"

In [5]:
def get_team_tabs_html(url):
    """Returns HTML object of the two teams' box score"""
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    teams_tab = soup.find("div", class_="StickyContainer").find_all("div", class_=BOXSCORE_KEY)
    print(teams_tab)
    return teams_tab

def get_players_html(tab):
    """Returns list of players' HTML object"""
    return tab.find("table", class_=PLAYER).find("tbody", class_=TABLE_BODY).find_all("tr", class_=TABLE_CONTENT)

def get_team_stats_data(tab):
    """Returns list of players data's HTML object"""
    return tab.find("div", class_="Table__ScrollerWrapper relative overflow-hidden") \
        .find("div", class_="Table__Scroller").find("table", class_="Table Table--align-right") \
        .find("tbody", class_=TABLE_BODY).find_all("tr", class_=TABLE_CONTENT)

def store_data_in_team(team, team_name, player_game_stats):
    """Store data to the team specified in input"""
    team.set_team_name(team_name)
    team.add_player_game_stats(player_game_stats)

def get_player_info(player_html):
    """Returns player id, full_name and position"""
    player_name_div = player_html.find("a", PLAYER_NAME)
    if not player_name_div:
        return
    player_id, player_full_name = player_name_div["href"].split("/")[-2:]
    print(player_name_div)
    player_url = 
    player_full_name = " ".join(map(lambda x: x.capitalize(), player_full_name.split("-")))
    player_position = player_html.find("span", class_=PLAYER_POSITION).text
    return player_id, player_full_name, player_position

def get_player_stats(player_data_html):
    """Returns list of stats, either or not the player played the game (DNP), if not why (DNP Reason)."""
    dnp_div = player_data_html.find("td", class_="tc td Table__TD")
    is_dnp = False
    dnp_reason = None
    if dnp_div:
        if 'dnp' in dnp_div.text.lower():
            is_dnp = True
            dnp_reason = dnp_div.text.split("-")[-1]
        stats = 17 * [0]
    else:
        stats = [elt.text for elt in player_data_html.find_all("td", class_="Table__TD Table__TD")]
    return stats, is_dnp, dnp_reason

def scrapping_stats(url):
    """Main scrapping function calling all sub-functions"""
    teams_tab = get_team_tabs_html(url)
    for team_number, team_tab in enumerate(teams_tab):
        print(team_tab.find("div", class_=TEAM_NAME))
        team_name = team_tab.find("div", class_=TEAM_NAME).text
        tab = team_tab.find("div", class_=TEAM_TAB).find("div", class_="flex")
        players = get_players_html(tab)
        datas = get_team_stats_data(tab)

        for player, data in zip(players, datas):
            player_info = get_player_info(player)
            if not player_info:
                continue
            player_id, player_full_name, player_position = player_info

            stats, is_dnp, dnp_reason = get_player_stats(data)
            #player_game_stats = PlayerGameStats(player_id, self.game_id, player_full_name,
            #                                    False, is_dnp, dnp_reason, player_position,
            #                                    Stats(stats))
            # self.players_stats.append(player_game_stats)

In [6]:
scrapping_stats("https://www.espn.com/nba/boxscore/_/gameId/401468390")

[<div class="Boxscore flex flex-column"><div class="Boxscore__Title flex items-center pt3 pb3 brdr-clr-gray-08"><div class="BoxscoreItem__TeamName h5">Toronto Raptors</div></div><div class="ResponsiveTable ResponsiveTable--fixed-left Boxscore flex flex-column"><div class="flex"><table class="Table Table--align-right Table--fixed Table--fixed-left" style="border-collapse:collapse;border-spacing:0"><colgroup class="Table__Colgroup"><col class="Table__Column"/></colgroup><thead class="Table__header-group Table__THEAD"></thead><tbody class="Table__TBODY"><tr class="Table__TR Table__TR--sm Table__even" data-idx="0"><td class="Table__customHeader Table__TD"><div class="Table__customHeader">starters</div></td></tr><tr class="Table__TR Table__TR--sm Table__even" data-idx="1"><td class="Table__TD"><div class="flex"><a class="AnchorLink truncate db external Boxscore__AthleteName" data-player-uid="s:40~l:46~a:3244" href="https://www.espn.com/nba/player/_/id/3244/thaddeus-young" tabindex="0">T. Yo

In [20]:
url = "https://www.espn.co.uk/nba/player/_/id/3135047/justise-winslow"
player_id = url.split("/")[-2]
player_id

'3135047'

In [17]:
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")
player_html = soup.find("div", class_="StickyContainer")

In [18]:
player_name_html = player_html.find("div", class_="PlayerHeader__Main_Aside min-w-0 flex-grow flex-basis-0").find("h1", class_="PlayerHeader__Name flex flex-column ttu fw-bold pr4 h2").find_all("span")
player_full_name = " ".join([p.text for p in player_name_html])
player_full_name

'Justise Winslow'

In [29]:
player_team_html = player_html.find("div", class_="PlayerHeader__Team n8 mt3 mb4 flex items-center mt3 mb4 clr-gray-01")
player_team_id = player_team_html.find("a")["href"].split("/")[-2].lower()
player_team_id

'por'

In [34]:
player_bio_html = player_html.find("div", class_="PlayerHeader__Bio pv5")
info_html = player_bio_html.find_all("li")

In [41]:
height, weight, birth_date, college = None, None, None, None
for info in info_html:
    print(height, weight, birth_date, college, sep=', ')
    title, result = info.find_all("div")[:2]
    if title.text == "HT/WT":
        height, weight = result.text.split(", ")
    elif title.text == "Birthdate":
        birth_date = result.text.split('(')[0]
    elif title.text == "College":
        college = result.text


None, None, None, None
1.98 m, 100 kg, None, None
1.98 m, 100 kg, 26/3/1996 , None
1.98 m, 100 kg, 26/3/1996 , Duke
1.98 m, 100 kg, 26/3/1996 , Duke
